In [2]:
import pandas as pd
import pydot

#getting the data
col_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'type']
data = pd.read_csv("iris.csv", skiprows=1, header=None, names=col_names)
data.head(10)

class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value
        
class DecisionTreeClassifier():

    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree ''' 
        
        X = [sample[:-1] for sample in dataset]
        Y = [sample[-1] for sample in dataset]

        num_samples = len(X)
        num_features = len(X[0])  # Assuming all samples have the same number of features

        
        # split until stopping conditions are met
        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"] > 0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                print("Decision Node Returned\nInfo Gain:", best_split["info_gain"], "Feature Index:", best_split["feature_index"])
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -9999
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = [sample[feature_index] for sample in dataset]
            possible_thresholds = sorted(set(feature_values))  # Use sorted set to get unique values
            print("Possuble Thresholds:",possible_thresholds)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                print("For thresh val:", threshold, "of feature",feature_index)
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if children are not null
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y = [sample[-1] for sample in dataset]
                    left_y = [sample[-1] for sample in dataset_left]
                    right_y = [sample[-1] for sample in dataset_right]

                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    print("Curr Info gain:", curr_info_gain)
                    # update the best split if needed
                    if curr_info_gain > max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                    print("Max Info Gain:", max_info_gain)
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = [sample for sample in dataset if sample[feature_index] <= threshold]

        dataset_right = [sample for sample in dataset if sample[feature_index] > threshold]
        
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="gini"):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode == "gini":
            gain = self.gini_index(parent) - (weight_l * self.gini_index(l_child) + weight_r * self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l * self.entropy(l_child) + weight_r * self.entropy(r_child))
        
        
        return gain
    
    def entropy(self, y):
        ''' function to compute entropy '''
        
        class_counts = {}
        for value in y:
            if value in class_counts:
                class_counts[value] += 1
            else:
                class_counts[value] = 1
                
        entropy = 0
        total_samples = len(y)
        for class_count in class_counts.values():
            p_cls = class_count / total_samples
            entropy += -p_cls * math.log2(p_cls)

        return entropy
    
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_counts = {}
        for value in y:
            if value in class_counts:
                class_counts[value] += 1
            else:
                class_counts[value] = 1
                
        gini = 1
        total_samples = len(y)
        for class_count in class_counts.values():
            p_cls = class_count / total_samples
            gini -= p_cls ** 2
            
        return gini
        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y_counts = {}
        for value in Y:
            if value in Y_counts:
                Y_counts[value] += 1
            else:
                Y_counts[value] = 1
                
        max_count = -1
        max_value = None
        for value, count in Y_counts.items():
            if count > max_count:
                max_count = count
                max_value = value
                
        return max_value
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print(col_names[tree.feature_index], "<=", tree.threshold, "Info gain:", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
            
    def save_tree_as_dot(self, filename="decision_tree.dot", tree=None):
        ''' function to save the decision tree as a DOT file '''

        if not tree:
            tree = self.root

        with open(filename, "w") as dotfile:
            dotfile.write("digraph DecisionTree {\n")
            self._generate_dot(tree, dotfile)
            dotfile.write("}\n")

    def _generate_dot(self, tree, dotfile, parent_node=None):
        ''' recursive function to generate DOT format text for the tree '''

        if tree.value is not None:
            node_label = str(tree.value)
        else:
            node_label = col_names[tree.feature_index] + " <= " + str(tree.threshold) + "\\nInfo gain: " + str(tree.info_gain)

        dotfile.write('  "{}" [label="{}"];\n'.format(id(tree), node_label))

        if parent_node is not None:
            dotfile.write('  "{}" -> "{}";\n'.format(id(parent_node), id(tree)))

        if tree.left:
            self._generate_dot(tree.left, dotfile, tree)

        if tree.right:
            self._generate_dot(tree.right, dotfile, tree)

    
    def fit(self, X, Y):
        ''' function to train the tree '''
        

        dataset = [list(x) + [y[0]] for x, y in zip(X, Y)]
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value is not None:
            return tree.value
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)



X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1, 1)

import random
def custom_train_test_split(X, Y, test_size=0.2, random_state=None):
    if random_state is not None:
        random.seed(random_state)

    num_samples = len(X)
    num_test = int(test_size * num_samples)

    indices = list(range(num_samples))
    random.shuffle(indices)

    test_indices = indices[:num_test]
    train_indices = indices[num_test:]

    X_train = [X[i] for i in train_indices]
    Y_train = [Y[i] for i in train_indices]
    X_test = [X[i] for i in test_indices]
    Y_test = [Y[i] for i in test_indices]

    return X_train, X_test, Y_train, Y_test

X_train, X_test, Y_train, Y_test = custom_train_test_split(X, Y, test_size=.2, random_state=41)

classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train, Y_train)
classifier.print_tree()
classifier.save_tree_as_dot("decision_tree.dot")

Y_pred = classifier.predict(X_test)
def custom_accuracy_score(y_true, y_pred):
    correct = sum(1 for true, pred in zip(y_true, y_pred) if true == pred)
    total = len(y_true)
    accuracy = correct / total
    return accuracy

acc = custom_accuracy_score(Y_test, Y_pred)
print("Accuracy:", acc)


Possuble Thresholds: [4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5.0, 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6.0, 6.1, 6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8, 6.9, 7.0, 7.1, 7.2, 7.3, 7.6, 7.7, 7.9]
For thresh val: 4.3 of feature 0
Curr Info gain: 0.005603408029878576
Max Info Gain: 0.005603408029878576
For thresh val: 4.4 of feature 0
Curr Info gain: 0.022993295019157034
Max Info Gain: 0.022993295019157034
For thresh val: 4.5 of feature 0
Curr Info gain: 0.028991545893719772
Max Info Gain: 0.028991545893719772
For thresh val: 4.6 of feature 0
Curr Info gain: 0.04762896825396812
Max Info Gain: 0.04762896825396812
For thresh val: 4.7 of feature 0
Curr Info gain: 0.060618686868686766
Max Info Gain: 0.060618686868686766
For thresh val: 4.8 of feature 0
Curr Info gain: 0.09525793650793635
Max Info Gain: 0.09525793650793635
For thresh val: 4.9 of feature 0
Curr Info gain: 0.10684391465462328
Max Info Gain: 0.10684391465462328
For thresh val: 5.0 of feature 0
Curr Info gain: 0.14297705314009657
Max

In [4]:
# Sample dataset with two features and labels
sample_data = [
    [2.0, 3.5, 'A'],
    [1.5, 4.0, 'A'],
    [4.5, 2.5, 'B'],
    [4.0, 3.0, 'B'],
    [3.0, 2.0, 'A'],
    [2.5, 2.0, 'A'],
    [4.5, 3.5, 'B'],
    [3.5, 3.5, 'B'],
    [1.0, 3.0, 'A'],
    [5.0, 2.0, 'B'],
    [3.5, 2.5, 'A'],
    [2.0, 2.5, 'A'],
    [4.0, 2.0, 'B'],
    [2.5, 3.0, 'A'],
    [1.5, 3.5, 'A'],
    [3.0, 3.0, 'A'],
    [3.5, 2.0, 'A'],
    [4.5, 3.0, 'B'],
    [2.0, 2.0, 'A'],
    [3.0, 3.5, 'A']
]

# Splitting the dataset into features (X) and labels (Y)
X_sample = [data[:2] for data in sample_data]
Y_sample = [data[2] for data in sample_data]

# Splitting the sample dataset into training and testing sets
X_train_sample, X_test_sample, Y_train_sample, Y_test_sample = custom_train_test_split(X_sample, Y_sample, test_size=0.2, random_state=42)
print("X Train:\n", X_train_sample)
print("Y Train:\n", Y_train_sample)
print("X Test:\n", X_test_sample)
print("Y Test:\n", Y_test_sample)
# Creating and fitting the decision tree classifier
classifier_sample = DecisionTreeClassifier(min_samples_split=2, max_depth=4)
classifier_sample.fit(X_train_sample, Y_train_sample)

# Printing the built tree
classifier_sample.print_tree()

# Making predictions on the test set
Y_pred_sample = classifier_sample.predict(X_test_sample)

# Calculating and printing accuracy
acc_sample = custom_accuracy_score(Y_test_sample, Y_pred_sample)
print("Accuracy:", acc_sample)


X Train:
 [[5.0, 2.0], [2.5, 3.0], [3.0, 3.0], [2.0, 2.0], [4.5, 3.5], [4.0, 2.0], [4.5, 3.0], [3.5, 2.5], [1.5, 4.0], [2.0, 2.5], [4.5, 2.5], [3.5, 2.0], [3.5, 3.5], [1.0, 3.0], [2.0, 3.5], [4.0, 3.0]]
Y Train:
 ['B', 'A', 'A', 'A', 'B', 'B', 'B', 'A', 'A', 'A', 'B', 'A', 'B', 'A', 'A', 'B']
X Test:
 [[3.0, 3.5], [2.5, 2.0], [1.5, 3.5], [3.0, 2.0]]
Y Test:
 ['A', 'A', 'A', 'A']
Possuble Thresholds: [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
For thresh val: 1.0 of feature 0
Curr Info gain: 0.02552083333333338
Max Info Gain: 0.02552083333333338
For thresh val: 1.5 of feature 0
Curr Info gain: 0.0546875
Max Info Gain: 0.0546875
For thresh val: 2.0 of feature 0
Curr Info gain: 0.17400568181818182
Max Info Gain: 0.17400568181818182
For thresh val: 2.5 of feature 0
Curr Info gain: 0.2296875
Max Info Gain: 0.2296875
For thresh val: 3.0 of feature 0
Curr Info gain: 0.2977430555555556
Max Info Gain: 0.2977430555555556
For thresh val: 3.5 of feature 0
Curr Info gain: 0.37968750000000007
Max 

In [2]:
pip install graphviz


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pydot

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
!dot -Tpng decision_tree -o dtree.png

Error: dot: can't open decision_tree: No such file or directory


In [1]:
for i in range(5):
    print(f"Iteration {i}")

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
